In [1]:
from osgeo import gdal
from osgeo import ogr
import rasterio as rio
import numpy as np
import os 
from pprint import pprint

In [4]:
data_path = 'data/'
output_path = 'raster_output/'
raster_norm_path = 'raster_norm/'
criteria_file_name = [('CuttingGrids', 'grid', False), ('Communityfeatures', 'weight', True), ('SBNFMortalityt', "tot_mortal", False), ('EgressRoutes', 'weight', True), ('PopulatedAreast', "pop_per_sq", False), ('DistCircuits', 'shape_leng', True)]


In [188]:
X_MIN, x_max, y_min, Y_MAX = 522556.47860572586, 528313.1113211802, 3780732.388856534, 3786279.2744338084
PIXEL_SIZE =10

In [9]:
X_RES = 575
Y_RES = 554
NO_DATA_VALUE = -9999

In [10]:
def calculate_distance(raster_ds):
   
    band = raster_ds.GetRasterBand(1)
    gdal.ComputeProximity(band, band, options=['VALUES=0', 'DISTUNITS=GEO', 'MAXDIST=100'])
    

In [11]:
def normalization(file_name, folder_path='raster_norm/'):
    input_im = gdal.Open(f'raster_output/{file_name}_raster.tif')
    input_im_band = input_im.GetRasterBand(1)
    stats = input_im_band.GetStatistics(False, True)
    min_value, max_value = stats[0], stats[1]
    output_im = (f'{folder_path}{file_name}_norm.tif')
    driver_tiff = gdal.GetDriverByName('GTiff')
    output_im_band = driver_tiff.CreateCopy(output_im, input_im, strict=0)
    input_im_band_ar = input_im.GetRasterBand(1).ReadAsArray()

    output_im_band_ar = abs(((input_im_band_ar - min_value) / (max_value - min_value))*9)
    output_im_band.GetRasterBand(1).WriteArray(output_im_band_ar)
    output_im_band = input_im_band_ar = None

In [12]:
def vector_to_raster(shpfile_path, output_path, options, distance=False):

    # making the shapefile as an object.
    vec_ds = ogr.Open(shpfile_path) 
    # getting layer information of shapefile.
    layer = vec_ds.GetLayer() 
    
    # get GeoTiff driver 
    driver = gdal.GetDriverByName('GTiff')

    # passing the filename, x and y direction resolution, no. of bands, new raster.
    new_raster = driver.Create(output_path, X_RES, Y_RES, 1, gdal.GDT_Byte)

    # transforms between pixel raster space to projection coordinate space.
    new_raster.SetGeoTransform((X_MIN, PIXEL_SIZE, 0, Y_MAX, 0, -PIXEL_SIZE))

    #get required raster band.
    band = new_raster.GetRasterBand(1)
    band.SetNoDataValue(NO_DATA_VALUE)
    band.FlushCache()


    #main conversion method
    gdal.RasterizeLayer(new_raster, [1], layer, options=options)
    if distance:
        calculate_distance(new_raster)
        
    

In [114]:
def grid_vector_to_raster(shpfile_path, output_path, options, distance=False):

    # making the shapefile as an object.
    vec_ds = ogr.Open(shpfile_path) 
    # getting layer information of shapefile.
    layer = vec_ds.GetLayer() 
    
    x_min, x_max, y_min, y_max = layer.GetExtent()
    
    x_res = int((x_max - x_min) // PIXEL_SIZE)
    y_res= int((y_max - y_min) // PIXEL_SIZE)
    print(x_res, y_res)
    
    # get GeoTiff driver 
    driver = gdal.GetDriverByName('GTiff')

    # passing the filename, x and y direction resolution, no. of bands, new raster.
    new_raster = driver.Create(output_path, x_res, y_res, 1, gdal.GDT_Byte)

    # transforms between pixel raster space to projection coordinate space.
    new_raster.SetGeoTransform((x_min, PIXEL_SIZE, 0, y_max, 0, -PIXEL_SIZE))

    #get required raster band.
    band = new_raster.GetRasterBand(1)
    band.SetNoDataValue(NO_DATA_VALUE)
    band.FlushCache()


    #main conversion method
    gdal.RasterizeLayer(new_raster, [1], layer, options=options)
    
        
    

### Convert from vector to raster

In [13]:
for file_name, attribute, distance  in criteria_file_name:
    vector_to_raster(f'{data_path}{file_name}.shp', f'{output_path}{file_name}_raster.tif', options=[f'ATTRIBUTE={attribute}'], distance=distance)
      

## Scale data

In [ ]:
for file_name, _, _ in criteria_file_name:
 normalization(file_name)

## Add Weight

In [15]:
weight = 0.2

In [16]:
rasters = dict()
for name in os.listdir(raster_norm_path):
   rasters[name.split('_')[0]] = gdal.Open(raster_norm_path+name)
    

In [17]:
arr_band = rasters['Communityfeatures'].GetRasterBand(1).ReadAsArray()
final_raster = arr_band*weight

In [18]:
for criteria in rasters.keys():
    if criteria != 'Communityfeatures':
        final_raster += rasters[criteria].GetRasterBand(1).ReadAsArray() * weight
    

# Zonal statiscts

In [169]:
final_ds = rio.open('final_raster.tiff')
grids_ds = rio.open('raster_norm/CuttingGrids_norm.tif')

In [183]:
final_arr = final_ds.read().copy().astype(float)
grids_arr = grids_ds.read().copy().astype(float)

In [184]:
final_arr.shape, grids_arr.shape

((1, 554, 575), (1, 554, 575))

In [142]:
for grid_number in range(len(grids_arr[0])):
    grids_arr[0][grid_number] = (np.average(grids_arr[0][grid_number]) + np.average(final_arr[0][grid_number]))/2

In [187]:
with rio.open('zonal_final_222244.tiff', 'w', 
              driver=grids_ds.driver,
              height=grids_ds.height,
              width=grids_ds.width,
              count=grids_ds.count,
              crs=grids_ds.crs,
              transform=grids_ds.transform,
              dtype=grids_ds.dtypes[0]) as dst:
    dst.write(grids_arr)
    

# Final Result 

In [19]:
final_ds = gdal.GetDriverByName('GTiff').Create("final_raster.tiff", X_RES, Y_RES, 1, gdal.GDT_Byte)

In [20]:
final_ds.SetGeoTransform(rasters['Communityfeatures'].GetGeoTransform())
final_ds.SetProjection(rasters['Communityfeatures'].GetProjection())

0

In [21]:
final_band = final_ds.GetRasterBand(1)
final_band.SetNoDataValue(NO_DATA_VALUE)
final_band.WriteArray(final_raster, 0, 0)
final_band.FlushCache()

In [22]:
for file in rasters.values():
    del file
    
del final_ds
del final_band